# Lecture 04.5 : Hyperparameter Optimization

written by SinsuSquid (bgkang) on 05 December 2024

흠, 원래는 Lecture 4로 마무리 하고 끝내려고 했고 심지어 Lecture 4 마무리에 오글거리는 작별인사까지 다 쎠놨는데 다시 와서 강의를 만들려니까 매우매우 부끄럽네요. 그럼에도 불구하고 굳이 강의를 추가하는데에는 그만큼 중요한 내용이라고 생각해주면 좋겠어요. (아 몰라 남사스러우니까 그만 쳐다봐요)
<br><br>
오늘 이야기할 내용은 hyperparameter optimization입니다. 앞서서 model 성능을 결정하는 중요한 요소중에 하나가 어떤 hyperparameter (e.g. Hidden Layer의 크기는 어떻게 할것인가?) 값을 사용하는지와 관련되어 있다고 말한적 있었죠. 그렇기에 주어진 target을 학습하기에 가장 적합한 hyperparameter를 optimize할 수 있다면 가장 최적의 model performance를 기대할 수 있겠죠? 만약 설정할 수 있는 hyperparameter가 끽해야 2~3개정도라면 optimization이 그렇게 어렵진 않을거에요. (hyperparameter 하나가 하나의 dimension이라 생각해봐요!) 하지만 optimizer의 learning rate, hidden layer의 size, learing rate의 decay 등등 이것저것 다 optimize하겠다고 하면 이를 알고리즘 없이 최적화 하는데는 매우 많은 시간이 필요할거에요. 생각해봐요, 어떤 hyperparmeter combination에 대한 performance를 알고싶다면 (속된말로 point를 하나 찍는다면) training/test cycle을 한바퀴 돌려야 한다는걸 의미하잖아요?
<br><br>
이런 이유로 "적은 노력을 들여" 가장 최적의 hyperparameter set을 탐색하기 위한 algorithm들이 존재합니다. 오늘 강의에서는 GridSeach, Bayesian Optimization (BO), Particle Swarm Optimization (PSO)를 소개할까 하는데, 사실 원리에 대한 설명은 document에 맡기고 사용법에 집중하여 진행해보도록 하겠습니다. Optimization이란 process 자체가 계산량이 어마무시하게 많은 편이기 때문에 실제로 code를 의미있는 정도로 실행시키긴 어려울 것 같아요. 조금 밍숭맹숭하긴 하겠지만 피와 살이 되는 지식이니까 잘 알아가길 바랄게요.

## Training Environment

처음부터 다시 생각해볼까요? 어떤 model의 어떤 target에 대한 performance를 비교하기 위해서는 hyperparameter set를 input으로 할 때 performance를 output으로 제시할 수 있는 function이 필요하다고 할 수 있겠네요. 저는 이를 위해서 `Cycle`이라 부르는 하나의 object를 형성하겠어요. 이 `Cycle` object에서 `run`이란 method를 실행시키면 training/test cycle을 한번 진행해서 test loss를 return하는 형태로 구현해볼까 합니다. 실제로 실행시켜보지 않을거라 `DataLoader`나 detail은 조금 생략했으니까 나중에 이 code를 보고 직접 할 일이 있을 때 참고하도록 하세요.

In [ ]:
# 실제로 optimization을 하긴 어려울 것 같으니, toy model부터 정의해볼게요.
import torch

class ToyModel(torch.nn.Module):
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        
        self.in_size, self.hidden_size, self.out_size = in_size, hidden_size, out_size
        self.linear00 = torch.nn.Linear(self.in_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.linear01 = torch.nn.Linear(self.hidden_size, self.out_size)

        self.reset_parameters()
        return

    def forward(self, x):
        x = self.linear00(x)
        x = self.relu(x)
        return self.linear01(x)

    def reset_parameters(self):
        self.linear00.reset_parameters()
        self.linear01.reset_parameters()
        return

이제 진짜 `Cycle`이란 class를 만들어볼까요?

In [ ]:
# 이 hyperparameter들이 optimization 대상이겠죠. dict로 나타내봅시다.
parameters = {
    # input과 output size는 optimization이 필요 없겠죠?
    'in_channel' : 10,
    'out_size' : 1,
    
    # 실제로 optimization이 되어야 할 부분입니다 
    'lr' : 1E-3, # optimizer의 초기 learning rate
    'factor' : 0.1, # ReduceLROnPlateau의 factor
    'hidden_size' : 32 # hidden layer 크기
}

class Cycle():
    def __init__(self, parameters, loaders):
        self.parameters = parameters
        
        # loaders - training/test DataLoader가 들어갈 자리입니다.
        sself.train_loader, self.test_loader = tuple(loaders)

        # self.parameter에 저장된 정보들이 어떻게 사용되는지 추적해보세요.
        self.model = ToyModel(
                        in_size = self.parameters['in_size'],
                        hidden_size = self.parameters['hidden_size'],
                        out_size = self.parameters['out_size'],
        )

        self.optimizer = torch.optim.Adam(
                            self.model.parameters(),
                            lr = self.parameters['lr']
        )
        self.criterion = torch.nn.MSELoss()
        self.scheduler = torch.optim_lr_scheduler.ReduceLROnPlateau(
                            self.optimizer,
                            factor = self.parameters['factor']
        )

        return

    # 기본적인 train/test/run method를 정의해볼게요 - 원래 하던거랑 똑같아요!
    def train(self):
        self.model.train()
        tot_loss = 0.0
        for batch, y in self.train_loader():
            self.optimizer.zero_grad()

            out = self.mode(batch)
            loss = self.criterion(out, y)
            loss.backward()
            self.optimizer.step()

            tot_loss += loss.item()

        return tot_loss / len(self.train_loader)
        
    def test(self):
        self.model.eval()
        tot_loss = 0.0
        for batch, y in self.test_loader():
            out = self.mode(batch)
            loss = self.criterion(out, y)

            tot_loss += loss.item()

        return tot_loss / len(self.test_loader)

    def run(self, epochs):
        self.training_loss = [] # learning curve 저장용
        for e in range(epochs):
            self.training_loss.append(self.train())
            print(f"Epoch : {e:05d} | Trn. Loss : {self.training_loss[-1]:.3f}")

        self.test_loss = self.test()
        print(f"\tTest Loss : {self.test_loss:.3f}")

        # run의 return값은 test_loss가 되도록 합시다.
        return self.test_loss

자, 여기까지 code를 주저리주저리 적다 보니 어려워보일수도 있을 것 같은데, 사실 지금까지 한거라고는 `parameters`와 `loaders`라는 input을 받아서 test loss를 출력하는 아주 간단한 형태의 function을 만들어준거밖에 없어요! (잊지 말아요. 컴퓨터에서의 함수는 내부에서 어떤 일이 일어나는지 몰라도 된다는 걸.)
<br><br>
그렇다면 이제 optimzation을 위해 필요한거라고는 이 `parameters`를 변화시키면서 training/test cycle을 수행했을 때 어떤 hyperparameter set에서 가장 최적의 test loss가 나타나는지만 확인하면 되는거에요!

# GridSearch

사실 GridSearch는 직접 code로 확인할 필요도 없고 개념만 알면 되는 정도입니다. 되게 쉽거든요!
<br>
![GridSearch](https://images.contentstack.io/v3/assets/bltb654d1b96a72ddc4/blt500831b25ec72372/660f40c8e838c8586360f703/SPC-Blog-Hyperparameter-optimization-2.jpg "sabrepc.com")
<br>
우리의 예시를 들어서 생각해보기로 할까요? 우리가 설정한 `parameters`라는 dict에서 실제로 hyperparameter에 해당하는 값은 `lr`, `factor`, `hidden_size`가 전부였어요. 각 요소에 대해 우리가 canditate solution을 다음과 같이 3개씩 골랐다고 생각해볼게요.

```python
lr = [1E-3, 1E-2, 1E-1]
factor = [1E-1, 5E-2, 1E-2]
hidden_size = [10, 25, 50]
```

GridSearch는 복잡할 것 없이, 이 candidate로부터 만들어질수 있는 3 $\times$ 3 $\times$ 3 = 27개의 조합에 대해 각각 test loss를 계산하고 이중 최적의 test loss를 갖는 solution을 찾는 방법이에요! 여기까지 읽고 위 그림을 다시 본다면, 왜 "Grid"라 부르는지 이해가 가죠? (잘 모르겠으면 지금 물어봐요, 지금.)
<br><br>
물론 매우매우 단순한 algorithm이지만, 생각보다 잘 working하는 방법이라는 점에서 꼭 한번 사용해야하는 방법이에요. 개념이 어렵지 않기 때문에 code에 대해서는 언급하지 않고 다음 algorithm으로 넘어가도록 할게요.

## Bayesian Optimization (BO)

일단 Python에서 BO를 사용하기 위해서는 [bayesian-optimization](https://github.com/bayesian-optimization/BayesianOptimization) 패키지를 설치해야 해요. Bayesian Optimization을 이해하기 위해서는 Bayes' theorm부터 해서 알아야 할 것이 상당히 많습니다. 사실 optimization은 "어떤 방법 써서 optimize 했다." 한줄로 설명이 끝나기 때문에 이론까지 알 필요까진 없다고 생각해요. 알고싶으면 Google한테 물어보세요. 

![Bayesian Optimization](https://upload.wikimedia.org/wikipedia/commons/0/02/GpParBayesAnimationSmall.gif "en.wikipedia.org")

BO의 핵심은 통계적 방법을 통해서 함수의 형태를 모르더라도 input이 주어졌을 떄 output이 어느 범위 안에 있을지 확률적으로 예상할 수 있다는 점입니다. 우리는 아무래도 loss를 낮추는데 관심이 있으니 (그림은 최댓값에 관심이 있네요), 그렇다면 어떤 input이 주어졌을 때 최소 loss를 기대할 수 있겠다를 알 수 있다는 뜻이니 이번엔 그 기대되는 input에서 직접 point를 찍어본다는 것입니다. point의 숫자가 많아질수록 더 많은 sample이 있다는걸 의미하니 다음 step의 통계적 추론이 더 정확하다는걸 의미하겠죠.
<br>
다시 말하지만, optimization에서 이론까지 모두 이해할 필요는 없어요.

구현에 관해 얘기해보자면, 우리에겐 'objective function'이라는게 필요합니다. 별건 없고, 어떤 함수를 최적화하길 원하는지 알려주면 되는 단계인데, 우린 이미 `Cycle`이란 class를 정의하면서 다 해줬던 일이에요. 노파심에 얘기하지만, `bayesian-optimization` package를 설치하는걸 잊지 말아요!

In [ ]:
# Objective Function
def objective_function(lr, factor, hidden_size):
    parameters = {
        'in_channel' : 10,
        'out_size' : 1,
        'lr' : lr,
        'factor' : factor,
        'hidden_size' : hidden_size
    }

    training = Training(parameters, loaders)

    # 왜인지는 모르겠지만 항상 최댓값을 찾는것만 지원하더라고요
    # 우리가 원하는건 최솟값이니까, loss에 minus를 붙이도록 하죠
    return -training.run(100) # epoch는 manual하게 정해볼게요

optimize를 위해 hyperparameter의 boundary를 정해줘야 하는데요 (마법같이 뿅 하고 최적 parameter를 알려주진 못해요) `pbound`라는 이름으로 정의해주게 됩니다.

In [ ]:
from bayes_opt import BayesianOptimization

pbounds = {
    'lr' : (1E-3, 1E-1),
    'factor' : (1E-2, 1E-1),
    'hidden_size' : (16, 512)
}

optimizer = BayesianOptimization(
    f = objective_function, # objective function이 들어가는 자리
    pbounds = pbounds,
    random_state = 42, # random seed
)

실제로 실행을 해볼 수 있다면 좋겠지만, 그러기엔 너무 양이 많아지니까 이쯤 마무리할게요.
```python
# 실제로 optimize 하는 부분
optimizer.maximize(
    init_points = 4, # 초반에 몇개의 sample을 만들것인가
    n_inter = 3 # iteration 횟수는?
)
print(optimizer.max) # optimized parameter 출력
```

BO는 수학적으로 next best solution을 찾아갈 수 있다는 것이 장점이지만, 이번 point를 찍어야지만 다음에 어떤 point를 찍을지를 결정할 수 있어요. 다시말해, 병렬화랑은 좀 결이 안맞는다는거죠. 하지만 꼭 병렬화가 필요없는 상황 (e.g. GPU 1개만 사용 가능할 떄)이라면 좋은 tool이니까, 이런게 존재한다는거 정도는 제발 기억해줘요.

## Particle Swarm Optimization (PSO)

PSO는 [Chemomile](https://github.com/SinsuSquid/Chemomile)에서도 사용되었던 방법이죠. (네, 홍보에요.) Python 환경에서 PSO의 적용을 위해 [pyswarms](https://github.com/ljvmiranda921/pyswarms)를 사용했습니다.

![Particle Swarm Optimization](https://upload.wikimedia.org/wikipedia/commons/e/ec/ParticleSwarmArrowsAnimation.gif "en.wikipedia.org")

PSO의 핵심은 hyperparameter space (위 그림에서는 x, y가 되겠네요) 위에다가 'agent'라는놈들을 많이 깔아놓은 다음, 각 'agent'가 hyperparameter space의 이곳저곳을 탐색하다보면 가장 optimal한 solution근처로 모이지 않겠느냐는 가정이에요. 각각의 'agent'를 GPU 1대라고 생각해볼게요. 무슨 소라냐고요? 병렬화에 유리하다는걸 말하고 싶었어요. 각 'agent'들이 이동하는데는 3개의 weight를 따라서 움직이는데, 이에 대한 자세한 설명들은 [PySwarm Document](https://pyswarms.readthedocs.io/en/latest/index.html)에 맡기도록 하겠습니다. 역시 사용법에 포커스하여 탐구해보도록 할게요.

사실 사용법만 보면 BO랑 판박이입니다. object function 정의해주고 optimizer instance 만든 다음 method 호출해주면 끝이에요.

In [ ]:
def objective_function(input_array): # array 형태의 input을 사용해야해요
    parameters = {
        'in_channel' : 10,
        'out_size' : 1,
        'lr' : input_array[0],
        'factor' : input_array[1],
        'hidden_size' : input_array[2]
    }

    training = Training(parameters, loaders)

    # 여긴 또 minimize밖에 안됩니다
    return training.run(100)

In [ ]:
import numpy as np
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fs

# lr, factor, hidden_size
bounds = (np.array([1E-3, 1E-2,  16]), # lower bounds
          np.array([1E-4, 1E-1, 512])) # upper bounds
# 이런 형태의 input만 지원하나봐요

# PSO에서 사용되는 parameter인데, 설명은 document를 찾아보세요
options = {'c1' : 0.5, 'c2' : 0.3, 'w' : 0.9}

optimizer = ps.single.GlobalBestPSO(
    n_particles = 10, # agent 수
    dimensions = 3, # hyperparameter 수
    bounds = bounds,
    options = options
)

```python
# 실제로 optimize하는 부분
stats = optimzer.optimize(
    objecive_function, # objecive function
    iters = 10, # iteration 횟수
    n_processes = 10, # 병렬화에 유리하다 했죠?
)
print(stats)
```

## Outtro

저도 이런 저런 자료들을 뒤져가며 GNN을 공부하긴 했지만, 특히 hyperparamter optimization같은 "technique"까지 포함해서 설명해주는 강의나 tutorial을 찾긴 어려웠던걸로 기억해요. 이 참에 한번 정리해두니 저한테도 좋은 일 같네요. 이제 진짜 들꽃반 여러분께 무엇을 더 가르쳐줄 수 있는지 당장 생각이 나지 않아요. 그만큼 제가 배워온 지식들을 최대한 많이 넘겨주고 싶었고, 더 빠르게 연구실에서 쓸모없는 사람이 되고싶은 바램이라고나 할까요? 여러분이 이번 프로젝트를 통해 배울 수 있었던 지식들이 나중에 어느 곳에서 사용되고 있을지를 기대하며 이번 수업 마무리하도록 할게요.